#### This notebook uses Pegasus Transformer Model for paraphrasing. it also implements a function to split the sentences correctly to be paraphrased to avoid plagarism detection 

In [1]:
!pip install transformers
!pip install sentence_splitter
!pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 44.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 65.8 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.0/45.0 kB 1.7 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 16.5 MB/s eta 0:00:00


In [ ]:
import torch
from transformers import PegasusForConditionalGeneration, PegasusTokenizer
model_name = 'tuner007/pegasus_paraphrase'
torch_device = 'cuda' if torch.cuda.is_available() else 'cpu'
tokenizer = PegasusTokenizer.from_pretrained(model_name)
model = PegasusForConditionalGeneration.from_pretrained(model_name).to(torch_device)

In [ ]:
def get_response(input_text,num_return_sequences):
  batch = tokenizer([input_text],truncation=True,padding='longest',max_length=60, return_tensors="pt").to(torch_device)
  translated = model.generate(**batch,max_length=60,num_beams=10, num_return_sequences=num_return_sequences, temperature=1.5)
  tgt_text = tokenizer.batch_decode(translated, skip_special_tokens=True)
  return tgt_text

In [ ]:
text = "Cutting carbon emissions to mitigate climate change and reducing the over-dependence on fossil fuels are listed as a priority in a great number of countries throughout the world."
len(text.split())

28

In [ ]:
get_response(text, 2)

['Reducing the over-dependence on fossil fuels and cutting carbon emissions are both listed as priorities by many countries.',
 'Reducing the over-dependence on fossil fuels and cutting carbon emissions are listed as priorities by many countries.']

In [4]:
context = "Cutting carbon emissions to mitigate climate change and reducing the over-dependence on fossil fuels are listed as a priority in a great number of countries throughout the world. As a result, investments in the energy sector are shifting towards renewable energies; among all the renewables, wind energy is experiencing remarkable growth in recent years. Globally, the wind energy sector has undergone two record years in new capacity installations – a total of 93 GW in 2020 [1] and 94 GW in 2021 [2]. However, these reports also emphasise the need to quadruple the current growth rate to stay on course for the ambitious goals of the Paris Agreement [3] and the European Union Green Deal [4], i.e., net-zero emissions by 2050."

In [5]:
from sentence_splitter import SentenceSplitter, split_text_into_sentences
splitter = SentenceSplitter(language='en')
sentence_list = splitter.split(context)
sentence_list

['Cutting carbon emissions to mitigate climate change and reducing the over-dependence on fossil fuels are listed as a priority in a great number of countries throughout the world.',
 'As a result, investments in the energy sector are shifting towards renewable energies; among all the renewables, wind energy is experiencing remarkable growth in recent years.',
 'Globally, the wind energy sector has undergone two record years in new capacity installations – a total of 93 GW in 2020 [1] and 94 GW in 2021 [2].',
 'However, these reports also emphasise the need to quadruple the current growth rate to stay on course for the ambitious goals of the Paris Agreement [3] and the European Union Green Deal [4], i.e., net-zero emissions by 2050.']

##### New Model Try

In [2]:
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
model = AutoModelForSeq2SeqLM.from_pretrained("ramsrigouthamg/t5-large-paraphraser-diverse-high-quality")
tokenizer = AutoTokenizer.from_pretrained("ramsrigouthamg/t5-large-paraphraser-diverse-high-quality")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#print ("device ",device)
model = model.to(device)

In [3]:
def generate_text(inp):
    context = inp
    text = "paraphrase: "+context + " </s>"
    encoding = tokenizer.encode_plus(text,max_length =128, padding=True, return_tensors="pt")
    input_ids,attention_mask  = encoding["input_ids"].to(device), encoding["attention_mask"].to(device)
    model.eval()
    diverse_beam_outputs = model.generate(
        input_ids=input_ids,attention_mask=attention_mask,
        max_length=128,
        early_stopping=True,
        num_beams=5,
        num_beam_groups = 5,
        num_return_sequences=5,
        diversity_penalty = 0.70)
    
    sent = tokenizer.decode(diverse_beam_outputs[0], skip_special_tokens=True,clean_up_tokenization_spaces=True)
    return sent

In [7]:
output = generate_text(sentence_list[1])
output

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2382: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(


'paraphrasedoutput: As a result, investments in the energy sector are shifting toward renewable energies; among all the renewables, wind energy is seeing record growth in recent years.'

In [8]:
sentence_list[1]

'As a result, investments in the energy sector are shifting towards renewable energies; among all the renewables, wind energy is experiencing remarkable growth in recent years.'